In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time
import json
from selenium import webdriver
import re
import random
import os
from selenium.webdriver.common.keys import Keys


In [2]:
main_url = "https://finance.yahoo.com/news"
driver = webdriver.Chrome()
driver.get(main_url)
sleep_sec= 4

In [3]:
# accepts the terms field
driver.find_element_by_xpath("//button[@type='submit']").click()
# loop as long there get new posts loaded on the site. all posts are inside the "li" tag
init_li=0
new_li = 1
while new_li > init_li:
    init_li = len(driver.find_elements_by_tag_name("li"))
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(sleep_sec)
    new_li=len(driver.find_elements_by_tag_name("li"))

In [4]:
time.sleep(sleep_sec)
# gets all urls of posts including ads
posts_url = [x.get_attribute("href") for x in driver.find_elements_by_xpath("//h3/a")]
# gets only the urls which lead to a yahoo finance news site
articles_url = [x for x in posts_url if "https://finance.yahoo.com/news" in x]
# driver.close()

In [5]:
print(len(posts_url),len(articles_url))

206 167


In [6]:
for x in articles_url: 
    print(x)

https://finance.yahoo.com/news/asia-stocks-track-u-rise-215720588.html
https://finance.yahoo.com/news/1-brazil-govt-cuts-2021-174528830.html
https://finance.yahoo.com/news/analysis-retail-investors-learn-love-174327658.html
https://finance.yahoo.com/news/fed-tools-showing-signs-stress-170702834.html
https://finance.yahoo.com/news/alden-poised-acquire-tribune-soon-153053354.html
https://finance.yahoo.com/news/municipal-market-sales-slacken-raising-173430866.html
https://finance.yahoo.com/news/this-week-in-bideonomics-bidens-best-revenue-idea-172941001.html
https://finance.yahoo.com/news/m-profit-seen-crashing-90-172915616.html
https://finance.yahoo.com/news/quake-yunnan-china-kills-two-172616285.html
https://finance.yahoo.com/news/k-c-southern-agrees-30-150300178.html
https://finance.yahoo.com/news/1-elon-musk-says-tesla-172224645.html
https://finance.yahoo.com/news/good-rx-ceo-dismisses-amazon-threat-customers-love-to-talk-to-a-person-in-a-white-coat-171844108.html
https://finance.yaho

In [7]:
dir_save="yahoo_finance_news"
base_path = os.getcwd()
articles_paths = os.path.join(base_path, dir_save)
dir_saved= os.path.join(articles_paths, dir_save)
try:
   os.mkdir(dir_save)
   print("directory created")
except:
   print("directory exists")

directory created


In [15]:
source=[]
title=[]
time=[]
text=[]
paths=[]
html_names = []
driver = webdriver.Chrome()
for counter, article in enumerate(articles_url[:3],1):
    driver.get(article)
#agreing to terms and conditions of webpage
    try:
        button_agree = driver.find_element_by_xpath("//button[@type='submit']").click()
        button_agree.click()
    except:
        pass
    # clicks button to see whole text
    try:
        driver.find_element_by_xpath("//button[contains( text(), 'Story continues')]").click()
    except:
        pass
    #title
    title_s = driver.find_element_by_tag_name('h1')
    #time
    time_s= driver.find_element_by_tag_name('time')
    #text content
    #text_s=driver.find_element_by_class_name('caas-body')
    #source
    sources=driver.find_element_by_css_selector('header>div>a>img') 
    source_s=sources.get_attribute('alt')
    source.append(source_s) 
    title.append(title_s.text) 
    time.append(time_s.text) 
    #text.append(text_s.text)
    # specifies name and path if the html
    html_name = f'{counter}_article.html'
    html_path = os.path.join(articles_paths, html_name)

    paths.append(f'\{dir_save}\{html_name}')
    html_names.append(html_name)
    # downloads page as html
    with open(html_path, 'w') as f:
        f.write(driver.page_source)
article_infos = pd.DataFrame({'Title':title,'Source':source,'Datetime': time, 'file':html_names, 'path':paths})
article_infos
driver.close()

In [17]:
print(article_infos.head())

                                               Title     Source  \
0  U.S. Stocks Push Higher; Bitcoin Resumes Decli...  Bloomberg   
1  UPDATE 1-Brazil govt cuts 2021 budget deficit ...    Reuters   
2  Analysis: Retail investors learn to love the c...    Reuters   

                     Datetime            file  \
0  Fri, May 21, 2021, 7:48 PM  1_article.html   
1  Fri, May 21, 2021, 7:45 PM  2_article.html   
2  Fri, May 21, 2021, 7:43 PM  3_article.html   

                                 path  
0  \yahoo_finance_news\1_article.html  
1  \yahoo_finance_news\2_article.html  
2  \yahoo_finance_news\3_article.html  


In [18]:
csv_path= os.path.join(articles_paths, "articles.csv")
article_infos.to_csv(csv_path)

In [76]:
# #setting up driver for one artice
# #article_url = 'https://finance.yahoo.com/news/1-scrubs-maker-figs-eyes-143010503.html'
# article_url = 'https://finance.yahoo.com/news/whats-store-dxc-technology-dxc-142302406.html'
# driver = webdriver.Chrome()
# driver.get(article_url)
# #agreing to terms and conditions of webpage
# button_agree = driver.find_elements_by_tag_name('Button')[0]
# button_agree.click()
# # link_text=driver.find_elements_by_tag_name('a')[0]
# # link_text.click()

In [78]:
# #Scraping for info
# #title
# title_s = driver.find_element_by_tag_name('h1')
# #time
# time_s= driver.find_element_by_tag_name('time')
# #text content
# text_s=driver.find_element_by_class_name('caas-body')
# #source
# sources=driver.find_element_by_css_selector('header>div>a>img')
# source_s=sources.get_attribute('alt')

# source=[]
# source.append(source_s)
# title=[]
# title.append(title_s.text)
# time=[]
# time.append(time_s.text)
# text=[]
# text.append(text_s.text)

# article_infos = pd.DataFrame({'Title':title,'Source':source,'Datetime': time})
# #({'Title':title,'Source':source, 'Datetime': time, 'Name': name, 'Path':path})
                              
# article_infos

,Title,Source,Datetime
0,What's in Store for DXC Technology (DXC) This ...,Zacks,"Fri, May 21, 2021, 4:23 PM"


'Reuters'